<a href="https://colab.research.google.com/github/VAIKUNDATH/assignment1/blob/main/assignment_5_hyperopt_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('/content/titanic (2).csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
x=df[['Pclass','SibSp','Parch','Fare']]
y=df.Survived

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=0)

In [12]:
from sklearn import svm
from sklearn.model_selection import KFold,cross_val_score
model=svm.SVC()
kf=KFold(n_splits=5)
score=cross_val_score(model,x,y,cv=kf)
print("Cross Validation Scores are {}".format(score))
print("Average Cross Validation Score :{}".format(score.mean()))

Cross Validation Scores are [0.62011173 0.66853933 0.66853933 0.66292135 0.70786517]
Average Cross Validation Score :0.6655953800765803


**Random Search**

In [14]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
param_grid={'C':[0.1,1,10,100,1000],
            'gamma':[1,0.1,0.01,0.001,0.0001],
            'kernel':['rbf']}
random_search=RandomizedSearchCV(estimator=SVC(),param_distributions=param_grid,verbose=1,n_jobs=1,
                                 n_iter=20)
random_result=random_search.fit(x_train,y_train)
print('Best Score:',random_result.best_score_*100)
print('Best Params:',random_result.best_params_)            


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Score: 69.3483870967742
Best Params: {'kernel': 'rbf', 'gamma': 0.001, 'C': 1000}


**hyperopt**

In [15]:
from hyperopt import tpe,hp,fmin,STATUS_OK,Trials

In [16]:
space={'C':hp.choice('C',[0.1,1,10,100,1000]),
       'gamma':hp.choice('gamma',[1,0.1,0.01,0.001,0.0001]),
       'kernel':hp.choice('kernel',['rbf'])}

In [17]:
def hyperparameter_tuning(params):
  clf=SVC(**params)
  acc=cross_val_score(clf,x_train,y_train,scoring="accuracy").mean()
  return{"loss":-acc,"status":STATUS_OK}

In [18]:
from sklearn.model_selection import cross_val_score
trials=Trials()
best=fmin(
    fn=hyperparameter_tuning,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)
print("Best: {}".format(best))

100%|██████████| 50/50 [00:34<00:00,  1.43it/s, best loss: -0.7029548387096775]
Best: {'C': 4, 'gamma': 2, 'kernel': 0}


In [19]:
trials.results

[{'loss': -0.6115612903225806, 'status': 'ok'},
 {'loss': -0.6597548387096774, 'status': 'ok'},
 {'loss': -0.6917290322580645, 'status': 'ok'},
 {'loss': -0.6854064516129033, 'status': 'ok'},
 {'loss': -0.6805806451612904, 'status': 'ok'},
 {'loss': -0.6677806451612903, 'status': 'ok'},
 {'loss': -0.6677806451612903, 'status': 'ok'},
 {'loss': -0.6468774193548387, 'status': 'ok'},
 {'loss': -0.6147870967741935, 'status': 'ok'},
 {'loss': -0.6468645161290323, 'status': 'ok'},
 {'loss': -0.7029548387096775, 'status': 'ok'},
 {'loss': -0.6645032258064516, 'status': 'ok'},
 {'loss': -0.685341935483871, 'status': 'ok'},
 {'loss': -0.6613548387096774, 'status': 'ok'},
 {'loss': -0.6805806451612904, 'status': 'ok'},
 {'loss': -0.6917677419354838, 'status': 'ok'},
 {'loss': -0.6115612903225806, 'status': 'ok'},
 {'loss': -0.6917290322580645, 'status': 'ok'},
 {'loss': -0.6917290322580645, 'status': 'ok'},
 {'loss': -0.6757935483870968, 'status': 'ok'},
 {'loss': -0.664541935483871, 'status': '